<a href="https://colab.research.google.com/github/Mohamm76/plant-classification-mlflow-tutoria/blob/main/notebooks/mlflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌸 Plant Classification with MLflow - Complete Tutorial
 ## تجارب شاملة لخوارزميات تصنيف النباتات
#
# **الهدف:** مقارنة 5 خوارزميات تعلم آلي باستخدام MLflow
#

In [1]:
# الخلية 0: إعداد بيئة التثبيت
import sys
import warnings
warnings.filterwarnings('ignore')

# تعطيل تحديثات pip التلقائية
import os
os.environ['PIP_NO_DEPS'] = 'True'

# الخلية 1: تثبيت مع خيارات آمنة
try:
    !pip install mlflow==2.9.2 scikit-learn pandas matplotlib seaborn \
        --disable-pip-version-check \
        --no-warn-conflicts \
        --quiet \
        --timeout 120
    print("✅ التثبيت اكتمل بنجاح!")
except Exception as e:
    print(f"⚠️ حدث خطأ: {e}")
    print("محاولة التثبيت بشكل منفصل...")
    !pip install mlflow==2.9.2 --quiet
    !pip install scikit-learn pandas matplotlib seaborn --quiet

✅ التثبيت اكتمل بنجاح!


In [2]:
# الخلية 2: ربط Google Drive
from google.colab import drive
drive.mount('/content/drive')

# مسار المشروع على Google Drive
project_path = '/content/drive/MyDrive/MLflow_Plant_Classification'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# الخلية 3: إعداد MLflow مع Google Drive
import mlflow # استيراد مكتبة MLflow لتتبع التجارب والتعلم الآلي
import os #استيراد مكتبة نظام التشغيل للتعامل مع الملفات والمجلدات

project_path = '/content/drive/MyDrive/MLflow_Plant_Classification'

# إنشاء مجلدات MLflow على Google Drive
#os.path.join(): دالة لإنشاء مسار مجلد بشكل صحيح لنظام التشغيل

#project_path: متغير يحتوي المسار الرئيسي للمشروع (مفترض أنه محدد مسبقاً)

#'mlflow_data': اسم المجلد الفرعي الذي سيحفظ بيانات MLflow
mlflow_dir = os.path.join(project_path, 'mlflow_data')
os.makedirs(mlflow_dir, exist_ok=True)

# تحديد مسارات التخزين
#URI لتخزين بيانات التتبع
mlflow_backend_uri = f"sqlite:///{mlflow_dir}/mlruns.db"
#URI لتخزين القطع الأثرية
mlflow_artifact_uri = f"file:{mlflow_dir}/mlartifacts"

# إعداد MLflow
mlflow.set_tracking_uri(mlflow_backend_uri)
mlflow.set_registry_uri(mlflow_artifact_uri)

print(f"✅ MLflow storage setup complete:")
print(f"   Backend: {mlflow_backend_uri}")
print(f"   Artifacts: {mlflow_artifact_uri}")


✅ MLflow storage setup complete:
   Backend: sqlite:////content/drive/MyDrive/MLflow_Plant_Classification/mlflow_data/mlruns.db
   Artifacts: file:/content/drive/MyDrive/MLflow_Plant_Classification/mlflow_data/mlartifacts


In [5]:
# الخلية 4: تحميل وتحضير البيانات
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# تحميل Iris dataset
#iris: كائن يحتوي على البيانات والمعلومات الوصفية
iris = load_iris()
#iris.data: مصفوفة (150×4) تحتوي على القياسات للنباتات
#iris.feature_names: أسماء الأعمد
#pd.DataFrame(): تحويل البيانات إلى جدول pandas
#X: DataFrame يحتوي على الميزات (المتغيرات المستقلة)
X = pd.DataFrame(iris.data, columns=iris.feature_names)
#iris.target: مصفوفة (150,) تحتوي على تصنيف كل نبتة (0, 1, 2)
#pd.Series(): تحويلها إلى سلسلة pandas
#name='target': إعطاء اسم للعمود
#y: Series تحتوي على التصنيفات (المتغير التابع)
y = pd.Series(iris.target, name='target')
#iris.target_names: أسماء الأنواع ['setosa', 'versicolor', 'virginica']
target_names = iris.target_names

#عرض معلومات البيانات
print("📊 Dataset Information:")
print(f"Features: {iris.feature_names}")
print(f"Target classes: {target_names}")
print(f"Dataset shape: {X.shape}")
print(f"Sample data:")
print(X.head())

# تقسيم البيانات
#random_state=42: للثبات والتكرارية (نفس التقسيم في كل مرة)
#stratify=y: الحفاظ على نسب الفئات في كلا المجموعتين
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

#عرض نتائج التقسيم
print(f"\n✅ Data split complete:")
print(f"   Training set: {X_train.shape[0]} samples")
print(f"   Testing set: {X_test.shape[0]} samples")


📊 Dataset Information:
Features: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Target classes: ['setosa' 'versicolor' 'virginica']
Dataset shape: (150, 4)
Sample data:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2

✅ Data split complete:
   Training set: 105 samples
   Testing set: 45 samples


In [6]:
# الخلية 5: إنشاء تجربة MLflow
#تعريف اسم التجربة
experiment_name = "Plant_Classification_Comparison"

# التحقق إذا التجربة موجودة
try:
  #حاولة إنشاء تجربة جديدة
    experiment_id = mlflow.create_experiment(
        experiment_name,
        # موقع حفظ الملفات (حددناه سابقاً في mlflow_artifact_uri)
        #إذا نجحت: تحصل على experiment_id جديد وتطبع ✅
        artifact_location=mlflow_artifact_uri
    )
    print(f"✅ Created new experiment: {experiment_name}")
except:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
    print(f"📁 Using existing experiment: {experiment_name}")

mlflow.set_experiment(experiment_name)

2026/02/10 16:52:04 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/10 16:52:04 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

✅ Created new experiment: Plant_Classification_Comparison


<Experiment: artifact_location='file:/content/drive/MyDrive/MLflow_Plant_Classification/mlflow_data/mlartifacts', creation_time=1770742326547, experiment_id='1', last_update_time=1770742326547, lifecycle_stage='active', name='Plant_Classification_Comparison', tags={}>

# الملفات التي تم إنشاؤها في الخليه السابقه على درايف

MLflow_Plant_Classification/
└── mlflow_data/
    ├── mlruns.db                    # قاعدة بيانات SQLite (تم إنشاؤها الآن)
    │   ├── experiments              # جدول التجارب
    │   ├── runs                     # جدول التشغيلات
    │   ├── metrics                  # جدول المقاييس
    │   ├── params                   # جدول المعلمات
    │   └── ... (23 جدول إضافي)
    │
    └── mlartifacts/                 # مجلد القطع الأثرية
        └── Plant_Classification_Comparison/  # مجلد التجربة

In [7]:
# الخلية 6: وظيفة مساعدة لتسجيل التجارب
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def log_classification_experiment(model, model_name, params, X_train, X_test, y_train, y_test):
    """
    تسجيل تجربة تصنيف كاملة في MLflow
    """
    #with: يضمن إغلاق التشغيل تلقائياً بعد الانتهاء
    with mlflow.start_run(run_name=model_name):

        # 1. تسجيل المعلمات
        mlflow.log_params(params)

        # 2. تدريب النموذج وقياس الوقت
        import time
        start_time = time.time()
        model.fit(X_train, y_train)
        training_time = time.time() - start_time

        # 3. التنبؤ والتقييم
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        # 4. حساب المقاييس
        metrics = {
            'accuracy_train': accuracy_score(y_train, y_pred_train),
            'accuracy_test': accuracy_score(y_test, y_pred_test),
            'precision_weighted': precision_score(y_test, y_pred_test, average='weighted'),
            'recall_weighted': recall_score(y_test, y_pred_test, average='weighted'),
            'f1_weighted': f1_score(y_test, y_pred_test, average='weighted'),
            'training_time_seconds': training_time
        }

        # 5. تسجيل المقاييس
        mlflow.log_metrics(metrics)

        # 6. إنشاء وحفظ مصفوفة الارتباك
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))

        # مصفوفة الارتباك
        cm = confusion_matrix(y_test, y_pred_test)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=target_names,
                    yticklabels=target_names,
                    ax=axes[0])
        axes[0].set_title('Confusion Matrix')
        axes[0].set_xlabel('Predicted')
        axes[0].set_ylabel('Actual')

        # مخطط أداء النموذج
        metrics_df = pd.DataFrame({
            'Metric': list(metrics.keys())[:5],
            'Value': list(metrics.values())[:5]
        })
        axes[1].barh(metrics_df['Metric'], metrics_df['Value'])
        axes[1].set_title('Model Performance Metrics')
        axes[1].set_xlim(0, 1)

        plt.tight_layout()

        # حفظ المؤشرات
        confusion_matrix_path = f"{model_name}_confusion_matrix.png"
        plt.savefig(confusion_matrix_path)
        mlflow.log_artifact(confusion_matrix_path)
        plt.close()

        # 7. حفظ النموذج
        mlflow.sklearn.log_model(model, f"{model_name}_model")

        # 8. إضافة tags
        mlflow.set_tag("algorithm_type", model_name)
        mlflow.set_tag("problem_type", "multiclass_classification")
        mlflow.set_tag("dataset", "iris")

        print(f"✅ Experiment logged: {model_name}")
        print(f"   Test Accuracy: {metrics['accuracy_test']:.4f}")
        print(f"   Training Time: {training_time:.2f} seconds")

        return metrics


In [8]:
# الخلية 7: تجربة خوارزميات متعددة
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# قائمة النماذج للمقارنة
models_to_test = [
    {
        'name': 'Logistic_Regression',
        'model': LogisticRegression(random_state=42, max_iter=1000),
        'params': {'solver': 'lbfgs', 'max_iter': 1000, 'multi_class': 'auto'}
    },
    {
        'name': 'Decision_Tree',
        'model': DecisionTreeClassifier(random_state=42),
        'params': {'max_depth': 5, 'min_samples_split': 2, 'criterion': 'gini'}
    },
    {
        'name': 'Random_Forest',
        'model': RandomForestClassifier(random_state=42, n_estimators=100),
        'params': {'n_estimators': 100, 'max_depth': None, 'random_state': 42}
    },
    {
        'name': 'SVM',
        'model': SVC(random_state=42, probability=True),
        'params': {'kernel': 'rbf', 'C': 1.0, 'probability': True}
    },
    {
        'name': 'KNN',
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': 5, 'weights': 'uniform'}
    }
]

print("🚀 Starting MLflow experiments with 5 algorithms...")
print("=" * 50)

results = {}
for model_info in models_to_test:
    print(f"\n🔬 Training {model_info['name']}...")
    metrics = log_classification_experiment(
        model=model_info['model'],
        model_name=model_info['name'],
        params=model_info['params'],
        X_train=X_train,
        X_test=X_test,
        y_train=y_train,
        y_test=y_test
    )
    results[model_info['name']] = metrics

print("\n" + "=" * 50)
print("✅ All experiments completed successfully!")

🚀 Starting MLflow experiments with 5 algorithms...

🔬 Training Logistic_Regression...
✅ Experiment logged: Logistic_Regression
   Test Accuracy: 0.9333
   Training Time: 0.05 seconds

🔬 Training Decision_Tree...
✅ Experiment logged: Decision_Tree
   Test Accuracy: 0.9333
   Training Time: 0.00 seconds

🔬 Training Random_Forest...
✅ Experiment logged: Random_Forest
   Test Accuracy: 0.8889
   Training Time: 0.15 seconds

🔬 Training SVM...
✅ Experiment logged: SVM
   Test Accuracy: 0.9556
   Training Time: 0.01 seconds

🔬 Training KNN...
✅ Experiment logged: KNN
   Test Accuracy: 0.9778
   Training Time: 0.00 seconds

✅ All experiments completed successfully!


In [9]:
# الخلية 9: تسجيل أفضل نموذج في Model Registry
print("\n🏆 Registering the best model...")

# العثور على أفضل نموذج
best_model_name = results_df.index[0]
best_accuracy = results_df.iloc[0]['accuracy_test']

print(f"Best model: {best_model_name} with accuracy: {best_accuracy:.4f}")

# البحث عن أفضل run
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

# البحث عن run لأفضل نموذج
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string=f"tags.algorithm_type = '{best_model_name}'",
    order_by=["metrics.accuracy_test DESC"]
)

if runs:
    best_run = runs[0]

    # تسجيل النموذج
    model_uri = f"runs:/{best_run.info.run_id}/{best_model_name}_model"

    registered_model = mlflow.register_model(
        model_uri=model_uri,
        name="Plant_Classifier_Best_Model"
    )

    print(f"✅ Model registered successfully!")
    print(f"   Model Name: {registered_model.name}")
    print(f"   Version: {registered_model.version}")
    print(f"   Run ID: {best_run.info.run_id}")

    # إضافة وصف للنموذج
    client.update_registered_model(
        name=registered_model.name,
        description=f"Best performing model ({best_model_name}) for plant classification with accuracy: {best_accuracy:.4f}"
    )

    # نقل النموذج إلى مرحلة Production
    client.transition_model_version_stage(
        name=registered_model.name,
        version=registered_model.version,
        stage="Production"
    )

    print("✅ Model transitioned to Production stage")
else:
    print("❌ Could not find the best run")



🏆 Registering the best model...


NameError: name 'results_df' is not defined

In [ ]:
# %% [markdown]
# # 🌸 Plant Classification with MLflow - Complete Tutorial
# ## تجارب شاملة لخوارزميات تصنيف النباتات
#
# **الهدف:** مقارنة 5 خوارزميات تعلم آلي باستخدام MLflow
#

# %% [code]
# الخلية 1: تثبيت المكتبات
!pip install mlflow==2.9.2
!pip install scikit-learn pandas numpy matplotlib seaborn

# %% [code]
# الخلية 2: ربط Google Drive
from google.colab import drive
drive.mount('/content/drive')

# مسار المشروع على Google Drive
project_path = '/content/drive/MyDrive/MLflow_Plant_Classification'

# %% [code]
# الخلية 3: إعداد MLflow مع Google Drive
import mlflow
import os

# إنشاء مجلدات MLflow على Google Drive
mlflow_dir = os.path.join(project_path, 'mlflow_data')
os.makedirs(mlflow_dir, exist_ok=True)

# تحديد مسارات التخزين
mlflow_backend_uri = f"sqlite:///{mlflow_dir}/mlruns.db"
mlflow_artifact_uri = f"file:{mlflow_dir}/mlartifacts"

# إعداد MLflow
mlflow.set_tracking_uri(mlflow_backend_uri)
mlflow.set_registry_uri(mlflow_artifact_uri)

print(f"✅ MLflow storage setup complete:")
print(f"   Backend: {mlflow_backend_uri}")
print(f"   Artifacts: {mlflow_artifact_uri}")

# %% [code]
# الخلية 4: تحميل وتحضير البيانات
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# تحميل Iris dataset
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name='target')
target_names = iris.target_names

print("📊 Dataset Information:")
print(f"Features: {iris.feature_names}")
print(f"Target classes: {target_names}")
print(f"Dataset shape: {X.shape}")
print(f"Sample data:")
print(X.head())

# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\n✅ Data split complete:")
print(f"   Training set: {X_train.shape[0]} samples")
print(f"   Testing set: {X_test.shape[0]} samples")

# %% [code]
# الخلية 5: إنشاء تجربة MLflow
experiment_name = "Plant_Classification_Comparison"

# التحقق إذا التجربة موجودة
try:
    experiment_id = mlflow.create_experiment(
        experiment_name,
        artifact_location=mlflow_artifact_uri
    )
    print(f"✅ Created new experiment: {experiment_name}")
except:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
    print(f"📁 Using existing experiment: {experiment_name}")

mlflow.set_experiment(experiment_name)

# %% [code]
# الخلية 6: وظيفة مساعدة لتسجيل التجارب
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def log_classification_experiment(model, model_name, params, X_train, X_test, y_train, y_test):
    """
    تسجيل تجربة تصنيف كاملة في MLflow
    """
    with mlflow.start_run(run_name=model_name):

        # 1. تسجيل المعلمات
        mlflow.log_params(params)

        # 2. تدريب النموذج وقياس الوقت
        import time
        start_time = time.time()
        model.fit(X_train, y_train)
        training_time = time.time() - start_time

        # 3. التنبؤ والتقييم
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        # 4. حساب المقاييس
        metrics = {
            'accuracy_train': accuracy_score(y_train, y_pred_train),
            'accuracy_test': accuracy_score(y_test, y_pred_test),
            'precision_weighted': precision_score(y_test, y_pred_test, average='weighted'),
            'recall_weighted': recall_score(y_test, y_pred_test, average='weighted'),
            'f1_weighted': f1_score(y_test, y_pred_test, average='weighted'),
            'training_time_seconds': training_time
        }

        # 5. تسجيل المقاييس
        mlflow.log_metrics(metrics)

        # 6. إنشاء وحفظ مصفوفة الارتباك
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))

        # مصفوفة الارتباك
        cm = confusion_matrix(y_test, y_pred_test)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=target_names,
                    yticklabels=target_names,
                    ax=axes[0])
        axes[0].set_title('Confusion Matrix')
        axes[0].set_xlabel('Predicted')
        axes[0].set_ylabel('Actual')

        # مخطط أداء النموذج
        metrics_df = pd.DataFrame({
            'Metric': list(metrics.keys())[:5],
            'Value': list(metrics.values())[:5]
        })
        axes[1].barh(metrics_df['Metric'], metrics_df['Value'])
        axes[1].set_title('Model Performance Metrics')
        axes[1].set_xlim(0, 1)

        plt.tight_layout()

        # حفظ المؤشرات
        confusion_matrix_path = f"{model_name}_confusion_matrix.png"
        plt.savefig(confusion_matrix_path)
        mlflow.log_artifact(confusion_matrix_path)
        plt.close()

        # 7. حفظ النموذج
        mlflow.sklearn.log_model(model, f"{model_name}_model")

        # 8. إضافة tags
        mlflow.set_tag("algorithm_type", model_name)
        mlflow.set_tag("problem_type", "multiclass_classification")
        mlflow.set_tag("dataset", "iris")

        print(f"✅ Experiment logged: {model_name}")
        print(f"   Test Accuracy: {metrics['accuracy_test']:.4f}")
        print(f"   Training Time: {training_time:.2f} seconds")

        return metrics

# %% [code]
# الخلية 7: تجربة خوارزميات متعددة
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# قائمة النماذج للمقارنة
models_to_test = [
    {
        'name': 'Logistic_Regression',
        'model': LogisticRegression(random_state=42, max_iter=1000),
        'params': {'solver': 'lbfgs', 'max_iter': 1000, 'multi_class': 'auto'}
    },
    {
        'name': 'Decision_Tree',
        'model': DecisionTreeClassifier(random_state=42),
        'params': {'max_depth': 5, 'min_samples_split': 2, 'criterion': 'gini'}
    },
    {
        'name': 'Random_Forest',
        'model': RandomForestClassifier(random_state=42, n_estimators=100),
        'params': {'n_estimators': 100, 'max_depth': None, 'random_state': 42}
    },
    {
        'name': 'SVM',
        'model': SVC(random_state=42, probability=True),
        'params': {'kernel': 'rbf', 'C': 1.0, 'probability': True}
    },
    {
        'name': 'KNN',
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': 5, 'weights': 'uniform'}
    }
]

print("🚀 Starting MLflow experiments with 5 algorithms...")
print("=" * 50)

results = {}
for model_info in models_to_test:
    print(f"\n🔬 Training {model_info['name']}...")
    metrics = log_classification_experiment(
        model=model_info['model'],
        model_name=model_info['name'],
        params=model_info['params'],
        X_train=X_train,
        X_test=X_test,
        y_train=y_train,
        y_test=y_test
    )
    results[model_info['name']] = metrics

print("\n" + "=" * 50)
print("✅ All experiments completed successfully!")

# %% [code]
# الخلية 8: مقارنة النتائج
import pandas as pd

# جمع النتائج في DataFrame
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('accuracy_test', ascending=False)

print("📊 Model Comparison Results:")
print("=" * 60)
print(results_df[['accuracy_test', 'f1_weighted', 'training_time_seconds']].round(4))

# إنشاء مخطط مقارنة
plt.figure(figsize=(12, 6))
ax = results_df['accuracy_test'].plot(kind='bar', color='skyblue')
plt.title('Model Accuracy Comparison', fontsize=16)
plt.xlabel('Model', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.ylim(0.8, 1.0)
plt.xticks(rotation=45, ha='right')

# إضافة القيم على الأعمدة
for i, v in enumerate(results_df['accuracy_test']):
    ax.text(i, v + 0.005, f'{v:.4f}', ha='center')

plt.tight_layout()

# حفظ المخطط
comparison_chart_path = "model_comparison.png"
plt.savefig(comparison_chart_path)
mlflow.log_artifact(comparison_chart_path)
plt.show()

# %% [code]
# الخلية 9: تسجيل أفضل نموذج في Model Registry
print("\n🏆 Registering the best model...")

# العثور على أفضل نموذج
best_model_name = results_df.index[0]
best_accuracy = results_df.iloc[0]['accuracy_test']

print(f"Best model: {best_model_name} with accuracy: {best_accuracy:.4f}")

# البحث عن أفضل run
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

# البحث عن run لأفضل نموذج
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string=f"tags.algorithm_type = '{best_model_name}'",
    order_by=["metrics.accuracy_test DESC"]
)

if runs:
    best_run = runs[0]

    # تسجيل النموذج
    model_uri = f"runs:/{best_run.info.run_id}/{best_model_name}_model"

    registered_model = mlflow.register_model(
        model_uri=model_uri,
        name="Plant_Classifier_Best_Model"
    )

    print(f"✅ Model registered successfully!")
    print(f"   Model Name: {registered_model.name}")
    print(f"   Version: {registered_model.version}")
    print(f"   Run ID: {best_run.info.run_id}")

    # إضافة وصف للنموذج
    client.update_registered_model(
        name=registered_model.name,
        description=f"Best performing model ({best_model_name}) for plant classification with accuracy: {best_accuracy:.4f}"
    )

    # نقل النموذج إلى مرحلة Production
    client.transition_model_version_stage(
        name=registered_model.name,
        version=registered_model.version,
        stage="Production"
    )

    print("✅ Model transitioned to Production stage")
else:
    print("❌ Could not find the best run")

# %% [code]
# الخلية 10: تشغيل MLflow UI باستخدام ngrok
print("\n🌐 Starting MLflow UI...")

# تثبيت ngrok
!pip install pyngrok

from pyngrok import ngrok

# فتح نفق لـ MLflow UI
public_url = ngrok.connect(5000)
print(f"✅ MLflow UI is available at: {public_url}")

# تشغيل MLflow UI في الخلفية
import threading
import subprocess

def run_mlflow_ui():
    subprocess.run(["mlflow", "ui", "--port", "5000",
                   "--backend-store-uri", mlflow_backend_uri,
                   "--artifacts-destination", mlflow_artifact_uri])

# بدء MLflow UI في thread منفصل
ui_thread = threading.Thread(target=run_mlflow_ui, daemon=True)
ui_thread.start()

print("📋 MLflow UI is running in the background")
print("⚠️  Note: The UI will stop when the Colab session ends")

# %% [code]
# الخلية 11: حفظ دفتر Colab على GitHub
print("\n💾 Saving notebook to GitHub...")

# إنشاء متطلبات المشروع
!pip freeze > requirements.txt

print("✅ Project requirements saved to requirements.txt")
print("\n📁 Project structure created on Google Drive:")
!ls -la "/content/drive/MyDrive/MLflow_Plant_Classification"

print("\n🎉 Project completed successfully!")
print("Next steps:")
print("1. Visit the MLflow UI link above to view experiments")
print("2. Check Google Drive for saved models and artifacts")
print("3. Push the code to GitHub from Colab")